## Reformat netCDF File for a Function Call

In [11]:
import opedia
import xarray as xr
import sys
import os
import numpy as np
import pandas as pd
import db
import common as com
import timeSeries as TS
import itertools as itt
from bokeh.io import output_notebook
from datetime import datetime, timedelta
import time
from math import pi
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import column
from bokeh.models import DatetimeTickFormatter
from bokeh.palettes import all_palettes
from bokeh.models import HoverTool
from bokeh.embed import components
import jupyterInline as jup
if jup.jupytered():
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm

### Testing Original Function

In [24]:
tables = ['tblSST_AVHRR_OI_NRT', 'tblAltimetry_REP']
variables = ['sst', 'sla']                                        # see catalog.csv  for the complete list of tables and variable names
startDate = '2016-03-29'
endDate = '2016-05-29'
lat1, lat2 = 25, 30
lon1, lon2 = -160, -155
depth1, depth2 = 0, 5
fname = 'XY'
exportDataFlag = False      # True if you you want to download data

plotXY(tables, variables, startDate, endDate, lat1, lat2, lon1, lon2, depth1, depth2, fname, exportDataFlag)


------------Time1:
[datetime.datetime(2016, 3, 29, 0, 0), datetime.datetime(2016, 3, 30, 0, 0), datetime.datetime(2016, 3, 31, 0, 0), datetime.datetime(2016, 4, 1, 0, 0), datetime.datetime(2016, 4, 2, 0, 0), datetime.datetime(2016, 4, 3, 0, 0), datetime.datetime(2016, 4, 4, 0, 0), datetime.datetime(2016, 4, 5, 0, 0), datetime.datetime(2016, 4, 6, 0, 0), datetime.datetime(2016, 4, 7, 0, 0), datetime.datetime(2016, 4, 8, 0, 0), datetime.datetime(2016, 4, 9, 0, 0), datetime.datetime(2016, 4, 10, 0, 0), datetime.datetime(2016, 4, 11, 0, 0), datetime.datetime(2016, 4, 12, 0, 0), datetime.datetime(2016, 4, 13, 0, 0), datetime.datetime(2016, 4, 14, 0, 0), datetime.datetime(2016, 4, 15, 0, 0), datetime.datetime(2016, 4, 16, 0, 0), datetime.datetime(2016, 4, 17, 0, 0), datetime.datetime(2016, 4, 18, 0, 0), datetime.datetime(2016, 4, 19, 0, 0), datetime.datetime(2016, 4, 20, 0, 0), datetime.datetime(2016, 4, 21, 0, 0), datetime.datetime(2016, 4, 22, 0, 0), datetime.datetime(2016, 4, 23, 0, 0), d

Loading BokehJS ...

### Testing NetCDF Compatible Function

In [50]:
xFile = xr.open_dataset('http://engaging-opendap.mit.edu:8080/thredds/dodsC/las/id-a1d60eba44/data_usr_local_tomcat_content_cbiomes_20190510_20_darwin_v0.2_cs510_darwin_v0.2_cs510_nutrients.nc.jnl')

tables = [xFile, xFile]
variables = ['O2','FeT']                                        # see catalog.csv  for the complete list of tables and variable names
startDate = '1992-04-30'
endDate = '1993-04-30'
lat1, lat2 = 25, 30
lon1, lon2 = -160, -155
depth1, depth2 = 0, 5
fname = 'XY'
exportDataFlag = False      # True if you you want to download data

xarrayPlotXY(tables, variables, startDate, endDate, lat1, lat2, lon1, lon2, depth1, depth2, fname, exportDataFlag)


------------Time1:
[cftime.DatetimeGregorian(1992, 5, 1, 12, 0, 0, 0, 4, 122)
 cftime.DatetimeGregorian(1992, 5, 4, 12, 0, 0, 0, 0, 125)
 cftime.DatetimeGregorian(1992, 5, 7, 12, 0, 0, 0, 3, 128)
 cftime.DatetimeGregorian(1992, 5, 10, 12, 0, 0, 0, 6, 131)
 cftime.DatetimeGregorian(1992, 5, 13, 12, 0, 0, 0, 2, 134)
 cftime.DatetimeGregorian(1992, 5, 16, 12, 0, 0, 0, 5, 137)
 cftime.DatetimeGregorian(1992, 5, 19, 12, 0, 0, 0, 1, 140)
 cftime.DatetimeGregorian(1992, 5, 22, 12, 0, 0, 0, 4, 143)
 cftime.DatetimeGregorian(1992, 5, 25, 12, 0, 0, 0, 0, 146)
 cftime.DatetimeGregorian(1992, 5, 28, 12, 0, 0, 0, 3, 149)
 cftime.DatetimeGregorian(1992, 5, 31, 12, 0, 0, 0, 6, 152)
 cftime.DatetimeGregorian(1992, 6, 3, 12, 0, 0, 0, 2, 155)
 cftime.DatetimeGregorian(1992, 6, 6, 12, 0, 0, 0, 5, 158)
 cftime.DatetimeGregorian(1992, 6, 9, 12, 0, 0, 0, 1, 161)
 cftime.DatetimeGregorian(1992, 6, 12, 12, 0, 0, 0, 4, 164)
 cftime.DatetimeGregorian(1992, 6, 15, 12, 0, 0, 0, 0, 167)
 cftime.DatetimeGregorian(1

Loading BokehJS ...

IndexError: single positional indexer is out-of-bounds

### Original Function

In [46]:
def plotXY(tables, variables, startDate, endDate, lat1, lat2, lon1, lon2, depth1, depth2, fname, exportDataFlag, marker='-', msize=15, clr='green'):
    p = []
    lw = 2
    w = 500
    h = 500
    TOOLS = 'pan,wheel_zoom,zoom_in,zoom_out,box_zoom, undo,redo,reset,tap,save,box_select,poly_select,lasso_select'
    tablePairs = list(itt.combinations(tables, 2))
    variablePairs = list(itt.combinations(variables, 2))
    for i in tqdm(range(len(tablePairs)), desc='overall'):
        t1, y1, y_std1 = TS.timeSeries(tablePairs[i][0], variablePairs[i][0], startDate, endDate, lat1, lat2, lon1, lon2, depth1, depth2)
        t2, y2, y_std2 = TS.timeSeries(tablePairs[i][1], variablePairs[i][1], startDate, endDate, lat1, lat2, lon1, lon2, depth1, depth2)
        
        if exportDataFlag:
            exportData(t1, y1, y_std1, t2, y2, y_std2, tablePairs[i][0], variablePairs[i][0], tablePairs[i][1], variablePairs[i][1], lat1, lat2, lon1, lon2, depth1, depth2)

        if len(y1[~np.isnan(y1)]) < 1:
            com.printTQDM('%d: No matching entry found: Table: %s, Variable: %s ' % (i+1, tablePairs[i][0], variablePairs[i][0]), err=True )
            # continue
        com.printTQDM('%d: %s retrieved (%s).' % (i+1, variablePairs[i][0], tablePairs[i][0]), err=False)    
        if len(y2[~np.isnan(y2)]) < 1:
            com.printTQDM('%d: No matching entry found: Table: %s, Variable: %s ' % (i+1, tablePairs[i][1], variablePairs[i][1]), err=True )
            # continue
        com.printTQDM('%d: %s retrieved (%s).' % (i+1, variablePairs[i][1], tablePairs[i][1]), err=False)     

        if len(t1)<1 or len(y1)<1 or len(t2)<1 or len(y2)<1:
            continue
        if (len(t1)-len(t2) != 0) or (len(y1)-len(y2) != 0):   
            continue
        output_notebook()
        p1 = figure(tools=TOOLS, toolbar_location="above", plot_width=w, plot_height=h)
        p1.xaxis.axis_label = variablePairs[i][0] + ' [' + db.getVar(tablePairs[i][0], variablePairs[i][0]).iloc[0]['Unit'] + ']'
        p1.yaxis.axis_label = variablePairs[i][1] + ' [' + db.getVar(tablePairs[i][1], variablePairs[i][1]).iloc[0]['Unit'] + ']'
        leg = variablePairs[i][0] + ' / ' + variablePairs[i][1]
        fill_alpha = 0.3       
        cr = p1.circle(y1, y2, fill_color="grey", hover_fill_color="firebrick", fill_alpha=fill_alpha, hover_alpha=0.6, line_color=None, hover_line_color="white", legend=leg, size=msize)
        #p1.line(y1, y2, line_color=clr, line_width=lw, legend=leg)
        p1.add_tools(HoverTool(tooltips=None, renderers=[cr], mode='hline'))    
        p.append(p1)
    dirPath = 'embed/'
    if not os.path.exists(dirPath):
        os.makedirs(dirPath)        
   # if not inline:      ## if jupyter is not the caller
   #     output_file(dirPath + fname + ".html", title="XY")
    show(column(p))
    return

### NetCDF Compatible Function

In [49]:
def xarrayPlotXY(tables, variables, startDate, endDate, lat1, lat2, lon1, lon2, depth1, depth2, fname, exportDataFlag, marker='-', msize=15, clr='green'):
    p = []
    lw = 2
    w = 500
    h = 500
    TOOLS = 'pan,wheel_zoom,zoom_in,zoom_out,box_zoom, undo,redo,reset,tap,save,box_select,poly_select,lasso_select'
    tablePairs = list(itt.combinations(tables, 2))
    variablePairs = list(itt.combinations(variables, 2))
    for i in tqdm(range(len(tablePairs)), desc='overall'):
        #t1, y1, y_std1 = TS.timeSeries(tablePairs[i][0], variablePairs[i][0], startDate, endDate, lat1, lat2, lon1, lon2, depth1, depth2)
        #t2, y2, y_std2 = TS.timeSeries(tablePairs[i][1], variablePairs[i][1], startDate, endDate, lat1, lat2, lon1, lon2, depth1, depth2)
        
        table = tables[i].sel(TIME = slice(startDate, endDate), LAT_C = slice(lat1, lat2), LON_C = slice(lon1, lon2), DEP_C = slice(depth1, depth2))
        y1 = table.variables[variables[0]][:,0,0,0].values.tolist()
        y2 = table.variables[variables[1]][:,0,0,0].values.tolist()
        t1 = table.variables['TIME'].values
        t2 = table.variables['TIME'].values
        
        #print tests
        print('------------Time1:')
        print(t1)
        print('------------Time2:')
        print(t2)
        print('------------------Var1:')
        print(y1)
        print('------------------Var2:')
        print(y2)
        
        if exportDataFlag:
            exportData(t1, y1, y_std1, t2, y2, y_std2, tablePairs[i][0], variablePairs[i][0], tablePairs[i][1], variablePairs[i][1], lat1, lat2, lon1, lon2, depth1, depth2)

        #if len(y1[~np.isnan(y1)]) < 1:
        #    com.printTQDM('%d: No matching entry found: Table: %s, Variable: %s ' % (i+1, tablePairs[i][0], variablePairs[i][0]), err=True )
            # continue
        #com.printTQDM('%d: %s retrieved (%s).' % (i+1, variablePairs[i][0], tablePairs[i][0]), err=False)    
        #if len(y2[~np.isnan(y2)]) < 1:
        #    com.printTQDM('%d: No matching entry found: Table: %s, Variable: %s ' % (i+1, tablePairs[i][1], variablePairs[i][1]), err=True )
            # continue
        #com.printTQDM('%d: %s retrieved (%s).' % (i+1, variablePairs[i][1], tablePairs[i][1]), err=False)     

        if len(t1)<1 or len(y1)<1 or len(t2)<1 or len(y2)<1:
            continue
        if (len(t1)-len(t2) != 0) or (len(y1)-len(y2) != 0):   
            continue
        output_notebook()
        p1 = figure(tools=TOOLS, toolbar_location="above", plot_width=w, plot_height=h)
        p1.xaxis.axis_label = variablePairs[i][0] + ' [' + db.getVar(tablePairs[i][0], variablePairs[i][0]).iloc[0]['Unit'] + ']'
        p1.yaxis.axis_label = variablePairs[i][1] + ' [' + db.getVar(tablePairs[i][1], variablePairs[i][1]).iloc[0]['Unit'] + ']'
        leg = variablePairs[i][0] + ' / ' + variablePairs[i][1]
        fill_alpha = 0.3       
        cr = p1.circle(y1, y2, fill_color="grey", hover_fill_color="firebrick", fill_alpha=fill_alpha, hover_alpha=0.6, line_color=None, hover_line_color="white", legend=leg, size=msize)
        #p1.line(y1, y2, line_color=clr, line_width=lw, legend=leg)
        p1.add_tools(HoverTool(tooltips=None, renderers=[cr], mode='hline'))    
        p.append(p1)
    dirPath = 'embed/'
    if not os.path.exists(dirPath):
        os.makedirs(dirPath)        
   # if not inline:      ## if jupyter is not the caller
   #     print('test')
   #     output_file(dirPath + fname + ".html", title="XY")
    show(column(p))
    return

#### Helper Functions